In [38]:
import pandas as pd
from pandas import ExcelWriter
from pandas import ExcelFile
from collections import defaultdict, Counter
import matplotlib.pyplot as plt
from matplotlib import style
from sklearn.linear_model import LinearRegression


import numpy as np
import math

from sklearn import preprocessing, svm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn import metrics



In [2]:
###CDC Data
oddf = pd.read_csv('../data/Pulled/CDCuse.csv')

In [3]:
oddf.head()

,State,Year,Month,Period,Indicator,Data Value,Percent Complete,Percent Pending Investigation,State Name,Footnote,Footnote Symbol,Predicted Value
0,AK,2015,April,12 month-ending,"Opioids (T40.0-T40.4,T40.6)",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
1,AK,2015,April,12 month-ending,Percent with drugs specified,88.0952381,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
2,AK,2015,April,12 month-ending,"Synthetic opioids, excl. methadone (T40.4)",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN
3,AK,2015,April,12 month-ending,Number of Drug Overdose Deaths,126,100,0.0,Alaska,Numbers may differ from published reports usin...,**,126
4,AK,2015,April,12 month-ending,"Natural, semi-synthetic, & synthetic opioids, ...",NaN,100,0.0,Alaska,Numbers may differ from published reports usin...,**,NaN


In [4]:
oddf = oddf[oddf['Indicator']=='Number of Drug Overdose Deaths']

In [5]:
#### Fixing dates
oddf['Year'] = oddf['Year'].astype(str)
oddf['Month'] = oddf['Month'].astype(str)
oddf['Date'] = oddf['Month'] + " " + oddf['Year']
oddf['Date'] = pd.to_datetime(oddf['Date'])

In [6]:
## Limiting columns
oddf = oddf[['State','State Name','Date','Year','Month','Data Value']]

In [7]:
##Droping NA and fixing datatypes
oddf = oddf.dropna(subset = ['Data Value'])
oddf = oddf.replace(',','', regex=True)
oddf['Data Value'] = oddf['Data Value'].astype(int)

In [8]:
oddf.head()

,State,State Name,Date,Year,Month,Data Value
3,AK,Alaska,2015-04-01,2015,April,126
12,AK,Alaska,2015-08-01,2015,August,124
27,AK,Alaska,2015-12-01,2015,December,121
44,AK,Alaska,2015-02-01,2015,February,127
55,AK,Alaska,2015-01-01,2015,January,126


In [9]:
oddf2 = oddf.groupby(['State Name', 'Year'],as_index=False)['Data Value'].sum()

In [10]:
oddf2.head(10)

,State Name,Year,Data Value
0,Alabama,2015,8818
1,Alabama,2016,8324
2,Alabama,2017,9551
3,Alabama,2018,9198
4,Alabama,2019,8643
5,Alabama,2020,10624
6,Alabama,2021,14096
7,Alabama,2022,5310
8,Alaska,2015,1472
9,Alaska,2016,1594


In [11]:
oddf2 = oddf2[oddf2['Year']!= '2022']  ##data for 2022 incomplete we can predict later

</h2> Loading census data

In [12]:
###2010-2019 Census dataset
cendf1 = pd.read_csv('../data/Pulled/Census_2010-2019.csv')

In [13]:
cendf1.head()

,SUMLEV,REGION,DIVISION,STATE,NAME,CENSUS2010POP,ESTIMATESBASE2010,POPESTIMATE2010,POPESTIMATE2011,POPESTIMATE2012,...,RDOMESTICMIG2019,RNETMIG2011,RNETMIG2012,RNETMIG2013,RNETMIG2014,RNETMIG2015,RNETMIG2016,RNETMIG2017,RNETMIG2018,RNETMIG2019
0,10,0,0,0,United States,308745538,308758105,309321666,311556874,313830990,...,0.000000,2.493773,2.682083,2.636187,2.921500,3.260435,3.252788,2.871957,2.153911,1.818059
1,20,1,0,0,Northeast Region,55317240,55318443,55380134,55604223,55775216,...,-5.254530,0.887909,-0.038355,-0.469783,-0.986097,-2.061965,-2.490484,-1.837048,-2.134447,-2.859713
2,20,2,0,0,Midwest Region,66927001,66929725,66974416,67157800,67336743,...,-2.365881,-0.963930,-0.973943,-0.006924,-0.762969,-1.388437,-1.241784,-0.557370,-0.922755,-1.111173
3,20,3,0,0,South Region,114555744,114563030,114866680,116006522,117241208,...,3.261349,5.130513,5.850458,5.292073,6.161501,7.277358,7.150074,6.198168,5.225519,5.203720
4,20,4,0,0,West Region,71945553,71946907,72100436,72788329,73477823,...,0.614245,2.723344,3.062896,3.162262,4.026429,4.987285,5.261078,4.021194,3.044951,2.312083


In [14]:
##Limiting :  The CDC dataset only had data since 2015, so we will match them up
cendf1 = cendf1[['NAME','POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017',
                'POPESTIMATE2018','POPESTIMATE2019','DEATHS2015','DEATHS2016',
                'DEATHS2017','DEATHS2018','DEATHS2019']]

In [15]:
###2020, 2021 dataset
cendf2 = pd.read_csv('../data/Pulled/Census_2020-2021.csv')

In [16]:
##Limiting
cendf2 = cendf2[['NAME','POPESTIMATE2020','POPESTIMATE2021','DEATHS2020','DEATHS2021']]

In [17]:
### Merging
cendf3 = pd.merge(cendf1,cendf2, how="inner",left_on ='NAME',right_on= 'NAME')

In [18]:
cendf3.head()

,NAME,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,DEATHS2015,DEATHS2016,DEATHS2017,DEATHS2018,DEATHS2019,POPESTIMATE2020,POPESTIMATE2021,DEATHS2020,DEATHS2021
0,United States,320635163,322941311,324985539,326687501,328239523,2699826,2703215,2788163,2824382,2835038,331501080,331893745,853448,3433943
1,Northeast Region,56034684,56042330,56059240,56046620,55982803,489209,480502,497919,503076,505588,57525633,57159838,148129,601644
2,Midwest Region,67860583,67987540,68126781,68236628,68329004,626084,619187,640407,627911,622854,68935174,68841444,193237,761784
3,South Region,120997341,122351760,123542189,124569433,125580448,1052226,1058081,1092315,1116929,1122130,126409007,127225329,337628,1366769
4,West Region,75742555,76559681,77257329,77834820,78347268,532307,545445,557522,576466,584466,78631266,78667134,174454,703746


In [19]:
##Removing regions to focus on states for ploting
reg_list = ['United States','Northeast Region','Midwest Region','South Region','West Region']
cendf3 = cendf3[~cendf3['NAME'].isin(reg_list)]

In [20]:
##Rearranging to keep in order
cendf3 = cendf3[['NAME','POPESTIMATE2015','POPESTIMATE2016','POPESTIMATE2017',
                'POPESTIMATE2018','POPESTIMATE2019','POPESTIMATE2020','POPESTIMATE2021',
                 'DEATHS2015','DEATHS2016',
                'DEATHS2017','DEATHS2018','DEATHS2019','DEATHS2020','DEATHS2021']]

In [21]:
cendf3 = cendf3.rename(columns = {'NAME':'State'})

In [22]:
cendf3.head()

,State,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021,DEATHS2015,DEATHS2016,DEATHS2017,DEATHS2018,DEATHS2019,DEATHS2020,DEATHS2021
5,Alabama,4852347,4863525,4874486,4887681,4903185,5024803,5039877,51876,51710,53195,53665,53879,16148,64868
6,Alaska,737498,741456,739700,735139,731545,732441,732673,4350,4447,4463,4664,4819,1384,5641
7,Arizona,6829676,6941072,7044008,7158024,7278717,7177986,7276316,52846,56032,56333,60127,60523,18340,75665
8,Arkansas,2978048,2989918,3001345,3009733,3017804,3012232,3025891,31318,31134,32340,31743,31322,9449,38257
9,California,38918045,39167117,39358497,39461588,39512223,39499738,39237836,253798,260121,265241,277578,282520,81738,332337


In [23]:
#combining opiod df and census, will need to pivot

oddf2.head()

,State Name,Year,Data Value
0,Alabama,2015,8818
1,Alabama,2016,8324
2,Alabama,2017,9551
3,Alabama,2018,9198
4,Alabama,2019,8643


In [24]:
oddf3 = oddf2.copy().pivot(index='State Name', columns='Year', values='Data Value').rename_axis(None, axis=0)

In [25]:
oddf3.head()

Year,2015,2016,2017,2018,2019,2020,2021
Alabama,8818,8324,9551,9198,8643,10624,14096
Alaska,1472,1594,1571,1442,1544,1591,2416
Arizona,14590,16403,17552,19683,21988,28466,32656
Arkansas,4616,4513,4775,5286,4739,5325,6775
California,56001,57154,58406,63121,70935,92812,128028


In [26]:
oddf3 = oddf3.reset_index()

In [27]:
oddf4 = oddf3.rename(columns = {'index':'State','2015':'Overdose 2015','2016':'Overdose 2016',
                               '2017':'Overdose 2017',
                               '2018':'Overdose 2018','2019':'Overdose 2019','2020':'Overdose 2020',
                               '2021':'Overdose 2021'})

In [28]:
oddf4.head()

Year,State,Overdose 2015,Overdose 2016,Overdose 2017,Overdose 2018,Overdose 2019,Overdose 2020,Overdose 2021
0,Alabama,8818,8324,9551,9198,8643,10624,14096
1,Alaska,1472,1594,1571,1442,1544,1591,2416
2,Arizona,14590,16403,17552,19683,21988,28466,32656
3,Arkansas,4616,4513,4775,5286,4739,5325,6775
4,California,56001,57154,58406,63121,70935,92812,128028


In [29]:
finaldf1 = pd.merge(cendf3,oddf4, how="inner",left_on ='State',right_on= 'State')

In [30]:
finaldf1.tail(8)

,State,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021,DEATHS2015,DEATHS2016,...,DEATHS2019,DEATHS2020,DEATHS2021,Overdose 2015,Overdose 2016,Overdose 2017,Overdose 2018,Overdose 2019,Overdose 2020,Overdose 2021
43,Texas,27470056,27914410,28295273,28628666,28995881,29217653,29527941,187837,188794,...,202786,60137,248605,30794,32557,34262,35934,35795,44082,56188
44,Utah,2981835,3041868,3101042,3153550,3205958,3281684,3337975,17269,17786,...,17443,5705,22167,7433,8333,7485,7836,6984,7081,7995
45,Vermont,625216,623657,624344,624358,623989,642495,645570,5912,5811,...,5634,1804,6884,1094,1363,1633,1614,1615,1717,2698
46,Virginia,8361808,8410106,8463587,8501286,8535519,8632044,8642274,65247,65329,...,69729,20881,84697,11819,14327,17070,17155,17822,22322,29678
47,Washington,7163657,7294771,7423362,7523869,7614893,7718785,7738692,53852,54708,...,58587,17643,70836,12643,13830,13223,14024,14446,17986,24125
48,West Virginia,1842050,1831023,1817004,1804291,1792147,1789798,1782959,22759,22459,...,22567,6995,27102,8230,9617,11836,10965,10079,13119,18063
49,Wisconsin,5760940,5772628,5790186,5807406,5822434,5892323,5895908,51244,51341,...,50393,16194,62985,10378,11926,13577,13474,13617,16886,19636
50,Wyoming,585613,584215,578931,577601,578759,577267,578803,4824,4790,...,4971,1500,6042,1229,1138,871,763,797,1075,1190


In [31]:
finaldf1['2015 OD deaths per 10,000 population'] = finaldf1['Overdose 2015'] / (finaldf1['POPESTIMATE2015']/10000)
finaldf1['2016 OD deaths per 10,000 population'] = finaldf1['Overdose 2016'] / (finaldf1['POPESTIMATE2016']/10000) 
finaldf1['2017 OD deaths per 10,000 population'] = finaldf1['Overdose 2017'] / (finaldf1['POPESTIMATE2017']/10000) 
finaldf1['2018 OD deaths per 10,000 population'] = finaldf1['Overdose 2018'] / (finaldf1['POPESTIMATE2018']/10000) 
finaldf1['2019 OD deaths per 10,000 population'] = finaldf1['Overdose 2019'] / (finaldf1['POPESTIMATE2019']/10000) 
finaldf1['2020 OD deaths per 10,000 population'] = finaldf1['Overdose 2020'] / (finaldf1['POPESTIMATE2020']/10000) 
finaldf1['2021 OD deaths per 10,000 population'] = finaldf1['Overdose 2021'] / (finaldf1['POPESTIMATE2021']/10000) 


In [32]:
finaldf1['2021 % increase in OD Deaths since 2015']=finaldf1['Overdose 2015']/ finaldf1['Overdose 2021']

In [33]:
finaldf1['2015 % of all deaths OD deaths'] = finaldf1['Overdose 2015']/finaldf1['DEATHS2015']
finaldf1['2016 % of all deaths OD deaths'] = finaldf1['Overdose 2016']/finaldf1['DEATHS2016']
finaldf1['2017 % of all deaths OD deaths'] = finaldf1['Overdose 2017']/finaldf1['DEATHS2017']
finaldf1['2018 % of all deaths OD deaths'] = finaldf1['Overdose 2018']/finaldf1['DEATHS2018']
finaldf1['2019 % of all deaths OD deaths'] = finaldf1['Overdose 2019']/finaldf1['DEATHS2019']
finaldf1['2020 % of all deaths OD deaths'] = finaldf1['Overdose 2020']/finaldf1['DEATHS2020']
finaldf1['2021 % of all deaths OD deaths'] = finaldf1['Overdose 2021']/finaldf1['DEATHS2021']


In [34]:
finaldf1[['State','2021 % of all deaths OD deaths']].tail()

,State,2021 % of all deaths OD deaths
46,Virginia,0.350402
47,Washington,0.340575
48,West Virginia,0.666482
49,Wisconsin,0.311757
50,Wyoming,0.196955


In [35]:
finaldf1 = finaldf1.rename(columns = {'Data Value':'Drug Overdose Deaths'})

In [36]:
finaldf1.to_csv('../Aggregated/finaldf1opfortableau.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../Aggregated/finaldf1opfortableau.csv'

In [37]:
finaldf1

,State,POPESTIMATE2015,POPESTIMATE2016,POPESTIMATE2017,POPESTIMATE2018,POPESTIMATE2019,POPESTIMATE2020,POPESTIMATE2021,DEATHS2015,DEATHS2016,...,"2020 OD deaths per 10,000 population","2021 OD deaths per 10,000 population",2021 % increase in OD Deaths since 2015,2015 % of all deaths OD deaths,2016 % of all deaths OD deaths,2017 % of all deaths OD deaths,2018 % of all deaths OD deaths,2019 % of all deaths OD deaths,2020 % of all deaths OD deaths,2021 % of all deaths OD deaths
0,Alabama,4852347,4863525,4874486,4887681,4903185,5024803,5039877,51876,51710,...,21.143117,27.968937,0.625568,0.169982,0.160975,0.179547,0.171397,0.160415,0.657914,0.217303
1,Alaska,737498,741456,739700,735139,731545,732441,732673,4350,4447,...,21.721886,32.975147,0.609272,0.338391,0.358444,0.352005,0.309177,0.320398,1.149566,0.428293
2,Arizona,6829676,6941072,7044008,7158024,7278717,7177986,7276316,52846,56032,...,39.657364,44.879854,0.446779,0.276085,0.292743,0.311576,0.327357,0.363300,1.552126,0.431587
3,Arkansas,2978048,2989918,3001345,3009733,3017804,3012232,3025891,31318,31134,...,17.677921,22.390099,0.681328,0.147391,0.144954,0.147650,0.166525,0.151299,0.563552,0.177092
4,California,38918045,39167117,39358497,39461588,39512223,39499738,39237836,253798,260121,...,23.496865,32.628711,0.437412,0.220652,0.219721,0.220200,0.227399,0.251080,1.135482,0.385235
5,Colorado,5450623,5539215,5611885,5691287,5758736,5784308,5812069,36253,36719,...,27.807994,35.944171,0.530898,0.305933,0.303303,0.319506,0.322214,0.313478,1.359104,0.435338
6,Connecticut,3587122,3578141,3573297,3571520,3565287,3600260,3605597,30375,30116,...,44.474566,49.115306,0.477215,0.278222,0.357816,0.404718,0.410133,0.437606,1.698345,0.470409
7,Delaware,941252,948921,956823,965479,973764,991886,1003384,8543,8505,...,56.004420,54.814508,0.434727,0.279878,0.320282,0.426852,0.473673,0.518828,2.097018,0.471900
8,District of Columbia,675400,685815,694906,701547,705749,690093,670050,4883,4852,...,82.191821,92.276696,0.234029,0.296334,0.532976,0.850331,0.678444,0.669368,4.089402,0.943681
9,Florida,20209042,20613477,20963613,21244317,21477737,21569932,21781128,191076,193671,...,37.721491,43.372868,0.390511,0.193075,0.249805,0.327526,0.294631,0.291466,1.279404,0.369673


In [62]:
###prepping a dataframe for Lr over several years will alsorename columns for simlicity

LRprepDf = finaldf1[['State','2015 OD deaths per 10,000 population','2016 OD deaths per 10,000 population','2017 OD deaths per 10,000 population',
                    '2018 OD deaths per 10,000 population','2019 OD deaths per 10,000 population','2020 OD deaths per 10,000 population',
                    '2021 OD deaths per 10,000 population']]

LRprepDf.columns = ['State', '2015','2016','2017','2018','2019','2020','2021']

In [63]:
LRprepDf.head()

,State,2015,2016,2017,2018,2019,2020,2021
0,Alabama,18.172649,17.115158,19.593861,18.818740,17.627318,21.143117,27.968937
1,Alaska,19.959376,21.498241,21.238340,19.615338,21.106015,21.721886,32.975147
2,Arizona,21.362653,23.631796,24.917632,27.497812,30.208621,39.657364,44.879854
3,Arkansas,15.500086,15.094059,15.909534,17.563020,15.703472,17.677921,22.390099
4,California,14.389469,14.592343,14.839489,15.995555,17.952673,23.496865,32.628711


In [64]:
### Create function to forcast out to 2025 if nothing changes


def linear_model(years_train,pop_train,years_test,pop_test,X,y):  ## LR
    
    regressor = LinearRegression()  
    
    regressor.fit(years_train, pop_train) #training the algorithm

    a = X[-1]
    a = a +1
    X = np.append(X, a)
    X = X.reshape(-1,1)
    
    pred1 = regressor.predict(X)
   
    pred1 = pred1[-1]
    acc = regressor.score(years_test,pop_test)

    
    return(pred1,acc)
    

def train_mod(train_mod_df, year_end): 
    import math
    j = train_mod_df.columns.values[-1]
    j = int(j)

    while j < year_end:
        j = j + 1
        new_train_df = train_df_load(train_mod_df, j)
    return(new_train_df)


def train_df_load(train_df, j_num):
    m_acc = []
    for i, row in train_df.iterrows():
        population = np.array(row[1:])
        years = list(train_df.columns.values)
        years.pop(0) ## removing the name
        if len(years)> len(population):   ## for each pass
            years.pop(-1)
        years = [int(k) for k in years] ## making them ints
        years = np.array(years) ## creating a years array
        X = years.reshape(-1,1)
        y = population.reshape(-1,1)
        years_train, years_test, pop_train, pop_test = train_test_split(X,y,test_size=1)
        gain,acc = linear_model(years_train,pop_train,years_test,pop_test,X,y)
        m_acc.append(acc)
        next_year = (gain) ### Projected Gain + last year
        next_year = next_year[0]
        # next_year = int(np.round(next_year))
        #next_year = math.trunc(next_year)
        year_ar = str(j_num)
        train_df.loc[i,year_ar] = next_year        
    return(train_df,m_acc)


In [65]:
demo_model,model_acc = train_mod(LRprepDf, 2025)

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:362: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try usin

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\Loganj2\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col

In [66]:
demo_model

,State,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
0,Alabama,18.172649,17.115158,19.593861,18.818740,17.627318,21.143117,27.968937,25.131145,26.500479,28.174995,29.216139
1,Alaska,19.959376,21.498241,21.238340,19.615338,21.106015,21.721886,32.975147,30.417427,27.780005,31.369073,33.017522
2,Arizona,21.362653,23.631796,24.917632,27.497812,30.208621,39.657364,44.879854,42.508711,49.574698,52.717392,55.313513
3,Arkansas,15.500086,15.094059,15.909534,17.563020,15.703472,17.677921,22.390099,21.592435,20.883093,23.085345,23.099723
4,California,14.389469,14.592343,14.839489,15.995555,17.952673,23.496865,32.628711,24.177251,32.635006,35.336433,35.976723
5,Colorado,20.348133,20.105737,21.465158,21.791205,21.292867,27.807994,35.944171,33.405898,32.672532,35.900522,39.044172
6,Connecticut,23.559277,30.116197,35.244761,35.900681,38.232546,44.474566,49.115306,52.383888,55.512284,59.195072,62.877859
7,Delaware,25.402336,28.706289,40.404547,46.215402,50.371548,56.004420,54.814508,63.784518,72.153602,75.795849,81.988606
8,District of Columbia,21.424341,37.706962,62.871813,51.457707,53.843505,82.191821,92.276696,100.171352,108.385468,120.651711,129.800302
9,Florida,18.255195,23.470082,31.217901,29.124024,28.727887,37.721491,43.372868,44.135232,47.936232,51.646737,55.539947
